# TAP Query History using python client

<p>Examples that use the proof-of-concept QueryHistory class to interact with the UWS jobs endpoint of a TAP service</p>
<p>Include fetching recent queries, and filtering by various params like phase, time & query text</p>
<p>Proof-of-concept source code can be found at <a href="https://github.com/stvoutsin/queryhistory">https://github.com/stvoutsin/queryhistory</a></p>


### Install queryhistory package

In [ ]:
pip install --user git+https://github.com/stvoutsin/queryhistory.git

### Import QueryHistory and helper libraries

In [ ]:
import requests
from contextlib import suppress
from datetime import datetime
import pandas as pd
import pyvo

In [ ]:
from queryhistory import QueryHistory

### Set TOKEN info & TAP jobs BASE_URL

In [ ]:
TOKEN = ""
TAP_URL = "https://data.lsst.cloud/api/tap"
JOB_URL = TAP_URL + "/async"

### Initialize QueryHistory class

In [ ]:
query_history_service = QueryHistory(base_url=JOB_URL, token=TOKEN)

### Get up to 5 queries that have phase=COMPLETED

Note that the Query objects that are returned contain all params available in the UWS job for that query. The string representation is limited for the purpose of readability

In [ ]:
queries = query_history_service.get_queries(limit=5, phase="COMPLETED")
pd.DataFrame(queries)

### Get my 5 most recent queries

In [ ]:
queries = query_history_service.get_queries(last=5)
pd.DataFrame(queries)

### Get last 3 queries after August 20th 2024

In [ ]:
queries = query_history_service.get_queries(after=datetime(2024, 8, 20), last=3)
pd.DataFrame(queries)

### Get queries that match this text: "SELECT TOP 10 * FROM ivoa.ObsCore" (limit=5, order by recency)

In [ ]:
filters = [
    lambda q: q.query_text == "SELECT TOP 10 * FROM ivoa.ObsCore",
]
queries = query_history_service.get_queries(limit=5, recent=True, filters=filters)
pd.DataFrame(queries)

### Run an ASYNC pyvo query

<p>We submit a query here to check that it appears in the query history</p>
<p>Because pyvo deletes the query after running it when using async, for the query to appear in the list it a trick here is to use a query that produces an error, as these do not get deleted</p>


In [ ]:
query = "SELECT TOP 1 INVALID from TAP_SCHEMA.schemas"
# Setup authorization
s = requests.Session()
s.headers["Authorization"] = "Bearer " + TOKEN
auth = pyvo.auth.authsession.AuthSession()
auth.credentials.set("lsst-token", s)
auth.add_security_method_for_url(TAP_URL, "lsst-token")
auth.add_security_method_for_url(TAP_URL + "/async", "lsst-token")
tap_service = pyvo.dal.TAPService(baseurl=TAP_URL, session=auth)
with suppress(pyvo.dal.DALQueryError):
    tap_service.run_async(query)


### Assert that it is the most recent query in our history

In [ ]:
assert query_history_service.get_queries(last=1)[-1].query_text == query

### Find any PENDING Jobs, if found start the first one

In [ ]:
pending_queries = query_history_service.get_queries(phase="PENDING")
query_count = len(pending_queries)
pd.DataFrame(pending_queries)

In [ ]:
if len(pending_queries) > 0:
    first_job = pending_queries[0]
    query_history_service.run_query(pending_queries[0].job_id) 

### Assert that we now have less PENDING jobs

In [ ]:
if len(pending_queries) > 0:
    assert query_count > len(query_history_service.get_queries(phase="PENDING"))